<h1> <center> 8 - Group vs Nationality </center> </h1>

**Author:** Sebastião Rosalino, nº 20230372

**Supervisors:** Flávio Pinheiro, António Curado

<div style="text-align: center;">
    <img src="../logos/nova_ims_logo.png" alt="Image 1" style="width: 15%; display: inline-block; margin: 0 10px;"/>
    <img src="../logos/european_parliament_logo.png" alt="Image 2" style="width: 15%; display: inline-block; margin: 0 10px;"/>
</div>

# Table of Contents

1. [Data Preparation](#Data-Preparation)
2. [EP Group vs Nationality Index](#EP-Group-vs-Nationality-Index)
   - 2.1. [All subjects](#All-subjects)
   - 2.2. [European citizenship](#European-citizenship)
   - 2.3. [Internal market, single market](#Internal-market-single-market)
   - 2.4. [Community policies](#Community-policies)
   - 2.5. [Economic, social and territorial cohesion](#Economic-social-and-territorial-cohesion)
   - 2.6. [Economic and monetary system](#Economic-and-monetary-system)
   - 2.7. [External relations of the Union](#External-relations-of-the-Union)
   - 2.8. [Area of freedom, security and justice](#Area-of-freedom-security-and-justice)
   - 2.9. [State and evolution of the Union](#State-and-evolution-of-the-Union)
   - 2.10. [Secondary Subjects](#Secondary-Subjects)
   - 2.11. [Tertiary Subjects](#Tertiary-Subjects)

In [27]:
# Import the necessary libraries and configurations
import pandas as pd
import warnings
import numpy as np

from datetime import datetime
from tqdm import tqdm
from scipy.stats import ttest_1samp

# Suppress all warnings
warnings.filterwarnings('ignore')

In [28]:
# Read all the datasets from their corresponding folders
group_memberships = pd.read_csv("../datasets/second_stage/3_group_memberships/group_memberships.csv")
bills = pd.read_csv("../datasets/second_stage/4_bills/bills.csv")
member_votes = pd.read_csv("../datasets/second_stage/5_member_votes/member_votes.csv")
primary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/primary_subject_bills.csv")
secondary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/secondary_subject_bills.csv")
tertiary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/tertiary_subject_bills.csv")

In [29]:
# A reminder of how the group_memberships dataset looks like
group_memberships

,group_code,start_date,end_date,id_mep,membership_length
0,RENEW,2019-07-02,2024-07-15,0,1840
1,RENEW,2019-07-02,2020-01-31,1,213
2,SD,2019-07-02,2022-10-02,2,1188
3,EPP,2023-07-07,2024-07-15,3,374
4,SD,2019-07-02,2024-07-15,4,1840
...,...,...,...,...,...
964,EPP,2024-03-26,2024-07-15,868,111
965,EPP,2024-04-02,2024-07-15,869,104
966,EPP,2024-04-02,2024-07-15,870,104
967,EPP,2024-04-02,2024-07-15,871,104


In [30]:
# Convert 'start_date' and 'end_date' columns to datetime
group_memberships['start_date'] = pd.to_datetime(group_memberships['start_date'])
group_memberships['end_date'] = pd.to_datetime(group_memberships['end_date'])

In [31]:
# A reminder of how the bills dataset looks like
bills

,timestamp,procedure_reference,id_bill,text
0,2020-06-17 23:24:52,2007/0181(NLE),200,ecmorocco euro-mediterranean aviation agreemen...
1,2021-12-14 14:49:19,2010/0112(NLE),786,recommendation draft council decision conclusi...
2,2020-06-17 23:24:52,2010/0180(NLE),201,eujordan agreement euro-mediterranean aviation...
3,2020-06-17 23:24:52,2010/0186(NLE),203,eugeorgia agreement common aviation area
4,2020-06-17 23:24:52,2012/0006(NLE),198,eumoldova common aviation area agreement
...,...,...,...,...
1898,2024-04-25 12:51:39,2024/2696(RSP),1902,joint motion resolution new allegation russian...
1899,2024-04-25 12:20:42,2024/2698(RSP),1898,european parliament resolution azerbaijan nota...
1900,2024-04-25 12:21:01,2024/2699(RSP),1901,european parliament resolution propose repeal ...
1901,2024-04-25 12:53:44,2024/2703(RSP),1900,joint motion resolution


In [32]:
# A reminder of how the member_votes dataset looks like
member_votes

,position,country_code,group_code,id_mep,id_bill
0,FOR,LUX,RENEW,0,0
1,FOR,GBR,RENEW,1,0
2,DID_NOT_VOTE,DEU,SD,2,0
3,FOR,DEU,SD,4,0
4,FOR,DEU,SD,5,0
...,...,...,...,...,...
1345669,DID_NOT_VOTE,PRT,EPP,868,1902
1345670,DID_NOT_VOTE,PRT,EPP,869,1902
1345671,DID_NOT_VOTE,PRT,EPP,870,1902
1345672,FOR,PRT,EPP,871,1902


## 1. Data Preparation
<a id="Data-Preparation"></a>

In [33]:
# Replace values in the 'position' column as follows:
# FOR for "supporting"
# AGAINST or ABSTENTION or DID_NOT_VOTE for "not_supporting" 
member_votes['position'] = member_votes['position'].replace({
    "FOR": "supporting",
    "AGAINST": "not_supporting",
    "ABSTENTION": "not_supporting",
    "DID_NOT_VOTE": "not_supporting"
})

# Visualize the updated dataset
member_votes

,position,country_code,group_code,id_mep,id_bill
0,supporting,LUX,RENEW,0,0
1,supporting,GBR,RENEW,1,0
2,not_supporting,DEU,SD,2,0
3,supporting,DEU,SD,4,0
4,supporting,DEU,SD,5,0
...,...,...,...,...,...
1345669,not_supporting,PRT,EPP,868,1902
1345670,not_supporting,PRT,EPP,869,1902
1345671,not_supporting,PRT,EPP,870,1902
1345672,supporting,PRT,EPP,871,1902


## 2. EP Group vs Nationality Index
<a id="EP-Group-vs-Nationality-Index"></a>

The **EP Group vs Nationality Index** measures the degree to which the voting behavior of MEPs from a given country aligns with ideological expectations based on EP group representation. It provides insight into the influence of national interests versus political group representation in bills.

### Methodology

1. **Observed Voting Behavior**:
   - For each country $c$, the observed voting behavior on a bill is summarized using the **Observed Rice Index**, defined as:
     $$
     R^{\text{obs}}_c = \frac{|S_c - N_c|}{S_c + N_c}
     $$
     where:
     - $S_c$ is the number of MEPs from country $c$ who voted in support of the bill.
     - $N_c$ is the number of MEPs from country $c$ who did not support the bill.
     - $S_c + N_c$ is the total number of MEPs from country $c$ who participated in the vote of the bill.

<br>

2. **Expected Voting Behavior**:
   - The expected voting behavior on a bill is calculated based on the proportional representation of political groups in the EP when the bill was being voted. Let $p_g$ be the proportion of MEPs in group $g$ when the bill was being voted, defined as:
     $$
     p_g = \frac{\text{Number of MEPs in group } g \text{ when the bill was being voted}}{\text{Total number of MEPs in the EP when the bill was being voted}}
     $$
   - Using these proportions, 100 MEPs are randomly sampled across all groups. Their voting positions are used to compute the **Expected Rice Index**:
     $$
     R^{\text{exp}} = \frac{|S^{\text{exp}} - N^{\text{exp}}|}{S^{\text{exp}} + N^{\text{exp}}}
     $$
     where $S^{\text{exp}}$ and $N^{\text{exp}}$ are the counts of supporting and opposing votes, respectively.

<br>

3. **Difference Calculation**:
   - The deviation for country $c$ in a bill is calculated as the difference between **observed** and **expected** **Rice Index**:
     $$
     \Delta R_c = R^{\text{obs}}_c - R^{\text{exp}}
     $$

4. **Aggregation**:
   - The deviations $\Delta R_c$ are averaged across all bills $B$ to compute the **EP Group vs Nationality Index** for each country:
     $$
     I_c = \frac{1}{B} \sum_{b=1}^{B} \Delta R_{c,b}
     $$
     where $\Delta R_{c,b}$ is the difference calculated for country $c$ on bill $b$, and $B$ is the total number of bills considered.

<br>


### **Statistical Test Procedure**

To assess whether $I_c$ (the **EP Group vs Nationality Index** for a given country) is statistically greater than 0, a one-sample, one-tailed $t$-test is performed:

1. **Null Hypothesis ($H_0$)**:
   - The mean difference $\mu_{\Delta R_c} = 0$: There is no significant deviation in voting behavior alignment for the country compared to the EP.

<br>

2. **Alternative Hypothesis ($H_1$)**:
   - The mean difference $\mu_{\Delta R_c} > 0$: MEPs from the country exhibit greater cohesiveness in voting behavior compared to the EP as a whole.

<br>

3. **Test Statistic**:
   - The $t$-statistic is calculated as:
     $$
     t = \frac{\overline{\Delta R_c}}{\frac{s_{\Delta R_c}}{\sqrt{n}}}
     $$
     where:
     - $\overline{\Delta R_c}$: Mean of the differences for the country.
     - $s_{\Delta R_c}$: Sample standard deviation of the differences.
     - $n$: Number of bills considered for the country.

<br>

4. **Significance Level**:
   - A $p$-value is computed for the one-tailed test. Results are considered statistically significant if $p < 0.05$.

<br>

5. **Filtering Statistically Significant Results**:
   - Only countries with $p < 0.05$ are considered in the results. This ensures that observed deviations are unlikely to have occurred by chance.

<br>

### Interpretation

The **EP Group vs Nationality Index** $I_c$ provides insight into the cohesiveness of voting behavior among MEPs from a given country compared to the EP as a whole:
- A **positive** $I_c$ indicates that MEPs from a given country exhibit greater cohesiveness in their voting behavior than the EP as a whole.
- A **negative** $I_c$ or $I_c = 0$ indicates that MEPs from a given country are less cohesive or equally cohesive in their voting behavior compared to the EP as a whole.

In [34]:
def calculate_rice_index(positions):
    """Calculate the Rice Index for a given set of positions."""
    supporting = positions.count("supporting")
    not_supporting = positions.count("not_supporting")
    total = len(positions)
    if total == 0:
        return 0
    return abs(supporting - not_supporting) / total


def compute_ep_group_vs_nationality_index(bills):

    def compute_index():

        # Set a seed for reproducibility
        np.random.seed(42)
    
        country_results = []

        # Iterate over each bill
        for _, bill in tqdm(bills.iterrows(), total=len(bills), desc="Processing bills"):
            
            timestamp = datetime.strptime(bill['timestamp'], "%Y-%m-%d %H:%M:%S")
            
            id_bill = bill['id_bill']

            # Filter for active group memberships at the bill's timestamp
            active_memberships = group_memberships[(group_memberships['start_date'] <= timestamp) & 
                                                   (group_memberships['end_date'] >= timestamp)]

            # Calculate group representation percentages
            group_counts = active_memberships['group_code'].value_counts()
            total_meps = group_counts.sum()
            group_probabilities = group_counts / total_meps

            # Filter for votes for the current bill
            bill_votes = member_votes[member_votes['id_bill'] == id_bill]

            # Calculate the Rice indices (Observed and Expected) for each country
            for country, group in bill_votes.groupby('country_code'):
                
                # Observed Rice Index of all MEPs from the country who participated in the bill
                observed_rice_index = calculate_rice_index(group['position'].tolist())

                # Calculate Expected Rice Index by sampling 100 random MEPs from all groups and calculating the Rice Index
                simulated_positions = []
                for _ in range(100):
                    sampled_group = np.random.choice(group_probabilities.index, p=group_probabilities.values)
                    sampled_votes = bill_votes[bill_votes['group_code'] == sampled_group]
                    if not sampled_votes.empty:
                        simulated_positions.append(np.random.choice(sampled_votes['position'].tolist()))

                expected_rice_index = calculate_rice_index(simulated_positions)

                # Subtract the Expected Rice Index from the Observed Rice Index. Negative values imply that Group alignment plays a stronger role than National alignment,
                # in the country's MEPs votes. Positive values imply that National alignment plays a stronger role than Group alignment, in the country's MEP votes
                difference = observed_rice_index - expected_rice_index
                country_results.append((country, difference))

        # Convert country_results to a pandas dataframe
        country_df = pd.DataFrame(country_results, columns=['country', 'difference'])

        # Perform t-tests to test if the differences are significantly bigger than 0 for each country
        test_results = []
        for country, group in country_df.groupby('country'):
            differences = group['difference'].values
            t_stat, p_value = ttest_1samp(differences, 0, alternative='greater')
            test_results.append({
                'Country': country,
                'EP Group vs Nationality Index': differences.mean(),
                'Significant': p_value < 0.05  # Mark as statistically significant if p-value < 0.05
            })

        # Convert test_results to a pandas dataframe
        test_results_df = pd.DataFrame(test_results)

        # Filter for statistically significant results only
        test_results_df = test_results_df[test_results_df['Significant']]

        # Drop the 'Significant' column
        test_results_df = test_results_df.drop(columns=['Significant'])

        # Sort by the EP Group vs Nationality Index in descending order
        test_results_df = test_results_df.sort_values(by='EP Group vs Nationality Index', ascending=False).reset_index(drop=True)

        return test_results_df

    # Compute the EP Group vs Nationality Index with statistical tests
    return compute_index()

### 2.1. All subjects
<a id="All-subjects"></a>

In [9]:
# Compute the EP Group vs Nationality Index for all subjects, regardless of primary subject
all_subjects_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(bills)

all_subjects_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 1903/1903 [27:05<00:00,  1.17it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.306327
1,SVN,0.282895
2,PRT,0.165751
3,LTU,0.165746
4,MLT,0.161597
5,IRL,0.159710
6,EST,0.152350
7,CYP,0.151291
8,ROU,0.145227
9,ESP,0.135477


### 2.2. European citizenship
<a id="European-citizenship"></a>

In [16]:
# Filter for bills whose primary subject is 1 (among other potential primary subjects)
primary_subject_bills_1 = primary_subject_bills[primary_subject_bills["primary_subject"] == 1]

In [17]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 1 (among other potential primary subjects)
primary_subject_1_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_1)

primary_subject_1_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 98/98 [01:27<00:00,  1.12it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.334286
1,IRL,0.242301
2,CYP,0.202041
3,PRT,0.201399
4,EST,0.193800
5,SVN,0.192653
6,DNK,0.178035
7,ESP,0.152438
8,SWE,0.146735
9,FIN,0.142951


### 2.3. Internal market, single market
<a id="Internal-market-single-market"></a>

In [18]:
# Filter for bills whose primary subject is 2 (among other potential primary subjects)
primary_subject_bills_2 = primary_subject_bills[primary_subject_bills["primary_subject"] == 2]

In [19]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 2 (among other potential primary subjects)
primary_subject_2_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_2)

primary_subject_2_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 195/195 [02:46<00:00,  1.17it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.337692
1,LUX,0.261573
2,LTU,0.175860
3,EST,0.170422
4,MLT,0.160308
5,ROU,0.151900
6,ESP,0.137470
7,BGR,0.135294
8,IRL,0.125157
9,DNK,0.112174


### 2.4. Community policies
<a id="Community-policies"></a>

In [20]:
# Filter for bills whose primary subject is 3 (among other potential primary subjects)
primary_subject_bills_3 = primary_subject_bills[primary_subject_bills["primary_subject"] == 3]

In [21]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 3 (among other potential primary subjects)
primary_subject_3_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_3)

primary_subject_3_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 597/597 [08:31<00:00,  1.17it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.292178
1,LUX,0.243897
2,MLT,0.177476
3,CYP,0.177108
4,EST,0.160053
5,PRT,0.139121
6,ROU,0.131557
7,LTU,0.129185
8,BGR,0.129039
9,HUN,0.126750


### 2.5. Economic, social and territorial cohesion
<a id="Economic-social-and-territorial-cohesion"></a>

In [22]:
# Filter for bills whose primary subject is 4 (among other potential primary subjects)
primary_subject_bills_4 = primary_subject_bills[primary_subject_bills["primary_subject"] == 4]

In [23]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 4 (among other potential primary subjects)
primary_subject_4_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_4)

primary_subject_4_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 394/394 [05:34<00:00,  1.18it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.313926
1,IRL,0.271070
2,SVN,0.242259
3,CYP,0.233215
4,PRT,0.225132
5,MLT,0.178613
6,ESP,0.163320
7,ROU,0.150033
8,LTU,0.146627
9,EST,0.140031


### 2.6. Economic and monetary system
<a id="Economic-and-monetary-system"></a>

In [24]:
# Filter for bills whose primary subject is 5 (among other potential primary subjects)
primary_subject_bills_5 = primary_subject_bills[primary_subject_bills["primary_subject"] == 5]

In [25]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 5 (among other potential primary subjects)
primary_subject_5_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_5)

primary_subject_5_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]


,Country,EP Group vs Nationality Index
0,SVN,0.430625
1,LUX,0.351667
2,BGR,0.216691
3,ROU,0.215249
4,PRT,0.202560
5,MLT,0.196250
6,LTU,0.183977
7,HUN,0.174851
8,EST,0.167560
9,HRV,0.161496


### 2.7. External relations of the Union
<a id="External-relations-of-the-Union"></a>

In [26]:
# Filter for bills whose primary subject is 6 (among other potential primary subjects)
primary_subject_bills_6 = primary_subject_bills[primary_subject_bills["primary_subject"] == 6]

In [27]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 6 (among other potential primary subjects)
primary_subject_6_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_6)

primary_subject_6_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 444/444 [06:11<00:00,  1.19it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.322102
1,SVN,0.299572
2,EST,0.177782
3,SWE,0.173342
4,ROU,0.164827
5,POL,0.162546
6,LTU,0.155758
7,DNK,0.146343
8,PRT,0.134076
9,MLT,0.128814


### 2.8. Area of freedom, security and justice
<a id="Area-of-freedom-security-and-justice"></a>

In [28]:
# Filter for bills whose primary subject is 7 (among other potential primary subjects)
primary_subject_bills_7 = primary_subject_bills[primary_subject_bills["primary_subject"] == 7]

In [29]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 7 (among other potential primary subjects)
primary_subject_7_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_7)

primary_subject_7_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 181/181 [02:35<00:00,  1.17it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.311308
1,SVN,0.232983
2,EST,0.189034
3,PRT,0.149034
4,ESP,0.143489
5,LTU,0.137800
6,SWE,0.137143
7,POL,0.136692
8,DNK,0.129550
9,MLT,0.128066


### 2.9. State and evolution of the Union
<a id="State-and-evolution-of-the-Union"></a>

In [30]:
# Filter for bills whose primary subject is 8 (among other potential primary subjects)
primary_subject_bills_8 = primary_subject_bills[primary_subject_bills["primary_subject"] == 8]

In [31]:
# Compute the EP Group vs Nationality Index for all bills whose primary subject is 8 (among other potential primary subjects)
primary_subject_8_ep_country_vs_nationality_index = compute_ep_group_vs_nationality_index(primary_subject_bills_8)

primary_subject_8_ep_country_vs_nationality_index

Processing bills: 100%|██████████| 569/569 [08:03<00:00,  1.18it/s]


,Country,EP Group vs Nationality Index
0,LUX,0.316719
1,SVN,0.269578
2,IRL,0.231990
3,LTU,0.198156
4,PRT,0.177625
5,LVA,0.176977
6,CYP,0.171646
7,ESP,0.158064
8,DNK,0.152592
9,MLT,0.142507


### 2.10. Secondary Subjects
<a id="Secondary-Subjects"></a>

In [38]:
# Iterate over each unique secondary subject and calculate the EP Country vs Nationality Index for each secondary subject
for secondary_subject in secondary_subject_bills['secondary_subject'].unique():
    print(f"Secondary subject: {secondary_subject}")
    subject_bills = secondary_subject_bills[secondary_subject_bills['secondary_subject'] == secondary_subject]
    results = compute_ep_group_vs_nationality_index(subject_bills)
    print(results)
    print('\n')
    print("--------------------")
    print('\n')

Secondary subject: 3.2


Processing bills: 100%|██████████| 89/89 [01:17<00:00,  1.15it/s]


   Country  EP Group vs Nationality Index
0      SVN                       0.241798
1      LUX                       0.212060
2      BGR                       0.145459
3      EST                       0.133077
4      IRL                       0.132468
5      ROU                       0.110090
6      POL                       0.104940
7      ESP                       0.100524
8      MLT                       0.087640
9      AUT                       0.082188
10     LTU                       0.072217
11     CYP                       0.064494
12     DNK                       0.063766
13     PRT                       0.059572
14     HUN                       0.057175
15     CZE                       0.051974
16     BEL                       0.045821
17     NLD                       0.045179
18     ITA                       0.044693
19     SVK                       0.032043


--------------------


Secondary subject: 6.4


Processing bills: 100%|██████████| 116/116 [01:42<00:00,  1.13it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.370057
1      SVN                       0.317241
2      EST                       0.192603
3      ROU                       0.176315
4      LTU                       0.161144
5      SWE                       0.159466
6      POL                       0.151733
7      MLT                       0.144138
8      DNK                       0.141385
9      ESP                       0.135955
10     PRT                       0.118079
11     HUN                       0.110082
12     BGR                       0.104899
13     LVA                       0.088707
14     SVK                       0.077005
15     FIN                       0.071309
16     AUT                       0.068382
17     HRV                       0.065272
18     CZE                       0.057299
19     IRL                       0.054854
20     NLD                       0.053413


--------------------


Secondary subject: 2.1


...

  Country  EP Group vs Nationality Index
0     EST                       0.384286
1     SVN                       0.355000
2     LUX                       0.353333
3     MLT                       0.346667
4     ESP                       0.313337
5     SWE                       0.290238
6     IRL                       0.173462
7     DEU                       0.143750


--------------------


Secondary subject: 8.6


Processing bills: 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


  Country  EP Group vs Nationality Index
0     SVN                       0.345000
1     PRT                       0.210476
2     ESP                       0.189915


--------------------


Secondary subject: 8.1


Processing bills: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]

Empty DataFrame
Columns: [Country, EP Group vs Nationality Index]
Index: []


--------------------




### 2.11. Tertiary Subjects
<a id="Tertiary-Subjects"></a>

In [39]:
# Iterate over each unique tertiary subject and calculate the EP Country vs Nationality Index for each tertiary subject
for tertiary_subject in tertiary_subject_bills['tertiary_subject'].unique():
    print(f"Tertiary subject: {tertiary_subject}")
    subject_bills = tertiary_subject_bills[tertiary_subject_bills['tertiary_subject'] == tertiary_subject]
    results = compute_ep_group_vs_nationality_index(subject_bills)
    print(results)
    print('\n')
    print("--------------------")
    print('\n')

Tertiary subject: 3.20.15


Processing bills: 100%|██████████| 26/26 [00:23<00:00,  1.12it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.253846
1      SVN                       0.176154
2      BGR                       0.159548
3      LTU                       0.143287
4      IRL                       0.140134
5      HUN                       0.111319
6      SWE                       0.109853
7      LVA                       0.103846
8      EST                       0.103114
9      AUT                       0.088785
10     ROU                       0.087547
11     SVK                       0.080676
12     ESP                       0.078855
13     DEU                       0.065032


--------------------


Tertiary subject: 6.40.05


Processing bills: 100%|██████████| 10/10 [00:08<00:00,  1.13it/s]


   Country  EP Group vs Nationality Index
0      LUX                       0.452000
1      SVN                       0.297000
2      LTU                       0.255091
3      ROU                       0.222598
4      CYP                       0.195333
5      AUT                       0.194632
6      BGR                       0.183529
7      PRT                       0.159524
8      EST                       0.129810
9      SWE                       0.117429
10     ESP                       0.102024


--------------------


Tertiary subject: 2.10.02


...

  Country  EP Group vs Nationality Index
0     EST                       0.644286
1     HRV                       0.480000
2     ESP                       0.322881
3     BGR                       0.272941
4     LTU                       0.242727
5     FIN                       0.072857


--------------------


Tertiary subject: 6.40.02


Processing bills: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


  Country  EP Group vs Nationality Index
0     NLD                       0.295172
1     DNK                       0.284286
2     POL                       0.220769


--------------------


Tertiary subject: 3.40.13


Processing bills: 100%|██████████| 2/2 [00:01<00:00,  1.10it/s]

  Country  EP Group vs Nationality Index
0     BEL                       0.369524
1     LUX                       0.266667
2     BGR                       0.089412


--------------------


